In [1]:
%display latex

## Métrica bi-invariante en $\operatorname{SO}(3)$

In [2]:
# Definimos el álgebra de Lie so(3)
structure_constants = {('e1','e2') : {'e3' : 1}, ('e1','e3') : {'e2' : -1} , ('e2','e3') : {'e1' : 1}}
hh.<e1,e2,e3> = LieAlgebra(SR, structure_constants) 

In [3]:
hh.basis()

Finite family {'e1': e1, 'e2': e2, 'e3': e3}

In [4]:
B = hh.killing_form_matrix()
B

[-2  0  0]
[ 0 -2  0]
[ 0  0 -2]

In [5]:
def inner_product_from_matrix(hh, g):
    return lambda x, y: x.to_vector() * g * y.to_vector()

def gram_schmidt(hh, inner_product):
    hh_on_basis = []
    for i, v in enumerate(hh.basis().values()):
        u = v
        for j in range(i):
            u -= inner_product(v, hh_on_basis[j]) * hh_on_basis[j]
        hh_on_basis.append( hh( ((1 / sqrt(inner_product(u, u))) * u).to_vector().canonicalize_radical() ))
    return hh_on_basis

def bracket_contracted(hh, inner_product, on_basis):
    hh_dim = hh.dimension()
    bc = {(i,j,k): None for i in range(hh_dim) for j in range(hh_dim) for k in range(hh_dim)}
    for index in bc.keys():
        i, j, k = index
        bc[index] = inner_product(on_basis[i].bracket(on_basis[j]), on_basis[k])
    return bc

def secc_curv(hh, metric, on_basis, i, j):
    hh_dim = hh.dimension()
    bc = bracket_contracted(hh, metric, on_basis)
    return sum(bc[i,j,k]*(-bc[i,j,k]+bc[j,k,i]+bc[k,i,j])/2 - (bc[i,j,k]-bc[j,k,i]+bc[k,i,j])*(bc[i,j,k]+bc[j,k,i]-bc[k,i,j])/4 - bc[k,i,i]*bc[k,j,j] for k in range(hh_dim))

def Ric_ob(hh, bc, j, h):
    hh_dim = hh.dimension()
    return sum(sum( bc[(i,k,i)] * (bc[(k,j,h)] + bc[(k,h,j)]) / 2 + bc[(i,k,h)] * bc[(i,k,j)] / 4 - bc[(i,j,k)] * bc[(k,h,i)] / 2 + bc[(i,k,i)] * bc[(j,h,k)] / 2 - bc[(i,j,k)] * bc[(i,h,k)] / 2 for i in range(hh_dim)) for k in range(hh_dim))

def Ric_on(hh, g, on_basis):
    hh_dim = hh.dimension()
    ip = inner_product_from_matrix(hh, g)
    on_basis = gram_schmidt(hh, ip)
    bc = bracket_contracted(hh, ip, on_basis)
    return matrix(hh_dim, hh_dim, lambda i, j: Ric_ob(hh, bc, i, j))

In [6]:
metric = inner_product_from_matrix(hh, -B)

In [7]:
on_basis = gram_schmidt(hh, metric)
on_basis

[1/2*sqrt(2)*e1, 1/2*sqrt(2)*e2, 1/2*sqrt(2)*e3]

In [8]:
# La métrica es Einstein
Ric_on(hh, -B, on_basis)

[1/4   0   0]
[  0 1/4   0]
[  0   0 1/4]

In [9]:
# Curvaturas seccionales en la base ortonormal
for i in range(3):
    for j in range(i+1,3):
        display(secc_curv(hh,metric, on_basis, i, j))

1/8

1/8

1/8

## Esferas de Berger


Los siguientes ejemplos son conocidos como las esferas de Berger, cuando uno considera el grupo de Lie $\widetilde{\operatorname{SO}(3)} \simeq \operatorname{SU}(2)$

In [10]:
#metric = inner_product_from_matrix(hh, -B)
r = var("r")
g = diagonal_matrix(SR,[1,1,r])
display(g)
metric = inner_product_from_matrix(hh, g)

[1 0 0]
[0 1 0]
[0 0 r]

In [11]:
on_basis = gram_schmidt(hh, metric)
on_basis

[e1, e2, 1/sqrt(r)*e3]

In [12]:
# No es Einstein (si r \neq 1), por lo tanto no es de curvatura constante
Ric_on(hh, g, on_basis).factor()

[-1/2*r + 1          0          0]
[         0 -1/2*r + 1          0]
[         0          0      1/2*r]

In [13]:
for i in range(3):
    for j in range(i+1,3):
        display(secc_curv(hh,metric, on_basis, i, j).factor())

-3/4*r + 1

1/4*r

1/4*r

Si $r < \frac43$ se tiene que $\kappa > 0$ (hay que probarlo!!!)